In [83]:
from solvers.adapted_ns.suot_l2 import SUOT_L2
from utils.utils import get_instance, primal_cost, primal_cost_full_quad
import numpy as np


EPS = 1e-15
n = 10
# np.random.seed(0)
a, b, C = get_instance(n)
tau = 1


graph = SUOT_L2(C, a, b, n, tau=tau)


In [84]:
# graph = SUOT_L2(C, a, b, n, tau=tau)
# graph.solve(n_max=10000)
# u, v = graph.f_array, graph.g_array

# perturb = np.random.rand(n)


# graph = SUOT_L2(C, a, b + perturb, n, tau=tau)
# graph.solve(n_max=10000)
# u_prime, v_prime = graph.f_array, graph.g_array

In [95]:
graph = SUOT_L2(C, a, b, n, tau=tau)
graph.solve(n_max=10000)
u, v = graph.f_array, graph.g_array

graph = SUOT_L2(C, a, b, n, tau=tau)
tp_graph = graph.solve(n_max=10000)
u, v = graph.f_array, graph.g_array


In [96]:
np.abs(b - tp_graph.sum(0) - v).sum()

np.float64(7.979727989493313e-17)

In [97]:
support = tp_graph > EPS
support.flatten()

array([ True, False, False, False, False,  True, False, False, False,
       False, False, False, False,  True, False, False,  True,  True,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False,  True, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
        True, False, False, False,  True, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False])

In [98]:
idx_rows, idx_cols = np.arange(n**2) // n, np.arange(n**2) % n

In [99]:
idx_rows_active, idx_cols_active = (
    idx_rows[support.flatten()],
    idx_cols[support.flatten()],
)
idx_rows_active.shape, idx_cols_active.shape

((15,), (15,))

In [100]:
# C.flatten()[support.flatten()]
idx_rows_active

array([0, 0, 1, 1, 1, 2, 3, 3, 4, 5, 6, 7, 8, 8, 9])

In [101]:
(
    C.flatten()[support.flatten()]
    - np.repeat(v[:, None], n, axis=1).flatten()[support.flatten()]
)

array([ 0.06670457,  0.08707374, -0.01922135,  0.01148758, -0.01429253,
       -0.04876664,  0.25876838,  0.12242416,  0.17310708,  0.05795432,
        0.15479363,  0.09558946,  0.19357698,  0.13565336,  0.19759531])

In [116]:
import numpy as np


def recover_u(C, v, mask):
    n = v.shape[0]
    u = np.full(n, np.nan)

    for i in range(n):
        idx = mask[i, :]  # colonnes valides pour la ligne i
        if np.any(idx):
            u[i] = np.mean(C[i, idx] - v[idx])

    return u


recover_u(C, v, support)
# C - (u[:, None] + v[None, :]) >= -EPS

array([ 0.06670457,  0.09145265,  0.03098973,  0.14830307,  0.07138057,
        0.07832348,  0.15479363, -0.00994703,  0.13700234,  0.29932182])

In [115]:
u = np.sum(support * (C - v[None, :]), axis=1) / np.sum(support, axis=1)
u


array([ 0.06670457,  0.09145265,  0.03098973,  0.14830307,  0.07138057,
        0.07832348,  0.15479363, -0.00994703,  0.13700234,  0.29932182])

In [46]:
# (u - u_prime >= 0).sum()

In [47]:
# np.sum(v - v_prime >= 0)

In [5]:
graph = SUOT_L2(C, a, 0 * b, n, tau=tau)
graph.solve(n_max=10000)
u, v = graph.f_array, graph.g_array
(u >= 0).sum(), (v <= 0).sum()

(np.int64(200), np.int64(200))

In [48]:
def projection_simplex_conditional(V, a, z=1):
    """
    Projection sur le simplex de masse z uniquement pour les lignes
    dont la somme est > a_i.

    Parameters
    ----------
    V : ndarray, shape (n_samples, n_features)
    a : ndarray, shape (n_samples,)
        Seuil sur la marginale ligne
    z : float
        Masse du simplex

    Returns
    -------
    V_proj : ndarray
    """
    V_proj = V.copy()

    # marginales lignes
    row_sums = V.sum(axis=1)

    # lignes à projeter
    mask = row_sums > a

    if not np.any(mask):
        return V_proj

    z = np.ones(len(V)) * z
    z = z[mask]
    V_sub = V[mask]
    n_features = V_sub.shape[1]

    # projection simplex standard
    U = np.sort(V_sub, axis=1)[:, ::-1]
    cssv = np.cumsum(U, axis=1) - z
    ind = np.arange(1, n_features + 1)

    cond = U - cssv / ind > 0
    rho = np.count_nonzero(cond, axis=1)
    theta = cssv[np.arange(len(V_sub)), rho - 1] / rho

    V_proj[mask] = np.maximum(V_sub - theta[:, None], 0)

    return V_proj


def projection_simplex(V, z=1, axis=None):
    """
    Projection of x onto the simplex, scaled by z:
        P(x; z) = argmin_{y >= 0, sum(y) = z} ||y - x||^2
    z: float or array
        If array, len(z) must be compatible with V
    axis: None or int
        axis=None: project V by P(V.ravel(); z)
        axis=1: project each V[i] by P(V[i]; z[i])
        axis=0: project each V[:, j] by P(V[:, j]; z[j])
    """
    if axis == 1:
        n_features = V.shape[1]
        U = np.sort(V, axis=1)[:, ::-1]
        z = np.ones(len(V)) * z
        cssv = np.cumsum(U, axis=1) - z[:, np.newaxis]
        ind = np.arange(n_features) + 1
        cond = U - cssv / ind > 0
        rho = np.count_nonzero(cond, axis=1)
        theta = cssv[np.arange(len(V)), rho - 1] / rho
        return np.maximum(V - theta[:, np.newaxis], 0)

    elif axis == 0:
        return projection_simplex(V.T, z, axis=1).T

    else:
        V = V.ravel().reshape(1, -1)
        return projection_simplex(V, z, axis=1).ravel()

In [49]:
n = 10
mat = 20 * np.random.rand(n, n)
mask = mat @ np.ones(n) > a
a = np.random.rand(n)
a /= a.sum()

In [50]:
mat_proj = projection_simplex_conditional(mat, a, a)
mat_proj2 = projection_simplex(mat, a, axis=1)


In [51]:
mask

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [52]:
(mat_proj @ np.ones(n) - a)[mask], mat_proj2 @ np.ones(n) - a


(array([-1.05471187e-15,  5.92556774e-02,  1.80791075e-01,  2.28170265e-01,
         1.19619934e-01,  1.92317175e-01,  2.24577904e-01,  1.46419950e-01,
        -3.48527439e-02,  2.24056239e-01]),
 array([-1.05471187e-15, -3.88578059e-16,  1.26287869e-15,  2.08166817e-17,
         5.13478149e-16, -1.41553436e-15,  1.71911096e-15, -1.66533454e-16,
        -6.10622664e-16, -1.63584424e-15]))

In [53]:
np.maximum([1, 2, 3], 2)

array([2, 2, 3])